In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------- Fusion Module 정의 -----------------
class FusionBlock(nn.Module):
    def __init__(self, sam2_in_channels, yolo_channels, fusion_method='concat'):
        """
        sam2_in_channels: SAM2 feature의 채널 수
        yolo_channels: YOLO feature의 채널 수 (fusion 후 목표 채널 수)
        fusion_method: 'add' 또는 'concat'
                         여기서는 'concat' 방식을 사용
        """
        super(FusionBlock, self).__init__()
        self.fusion_method = fusion_method
        # SAM2 feature의 채널 수를 YOLO feature와 맞추기 위한 1x1 Conv
        self.adapter = nn.Conv2d(sam2_in_channels, yolo_channels, kernel_size=1)
        if self.fusion_method == 'concat':
            # Concatenation 후 채널 수 축소를 위한 1x1 Conv
            self.fusion_conv = nn.Conv2d(yolo_channels * 2, yolo_channels, kernel_size=1)
    
    def forward(self, yolo_feat, sam2_feat):
        """
        yolo_feat: YOLO의 feature map, shape: [B, C, H, W]
        sam2_feat: SAM2의 feature map (채널 및 spatial 크기가 다를 수 있음)
        """
        # 1x1 Conv를 통해 SAM2 feature의 채널 수를 맞춤
        adapted_feat = self.adapter(sam2_feat)
        # 만약 spatial 크기가 다르다면 YOLO feature의 크기에 맞게 조정
        if adapted_feat.shape[-2:] != yolo_feat.shape[-2:]:
            adapted_feat = F.interpolate(adapted_feat, size=yolo_feat.shape[-2:], mode='bilinear', align_corners=False)
        
        # Concatenation 방식: 두 feature map을 채널 차원에서 이어 붙이고, fusion_conv로 채널 축소
        if self.fusion_method == 'concat':
            fused = torch.cat([yolo_feat, adapted_feat], dim=1)
            fused = self.fusion_conv(fused)
        elif self.fusion_method == 'add':
            fused = yolo_feat + adapted_feat
        else:
            raise ValueError("fusion_method는 'add' 또는 'concat' 이어야 합니다.")
        return fused

# 여러 scale에서 fusion을 동시에 수행하는 모듈
class YOLOSAM2Fusion(nn.Module):
    def __init__(self, fusion_configs, fusion_method='concat'):
        """
        fusion_configs: 각 scale별 (SAM2 채널, YOLO 채널) 튜플 리스트
                        예: [(32, 32), (64, 64), (256, 256)]
        fusion_method: 'add' 또는 'concat'
        """
        super(YOLOSAM2Fusion, self).__init__()
        self.fusion_blocks = nn.ModuleList(
            [FusionBlock(sam2_in, yolo_ch, fusion_method) for sam2_in, yolo_ch in fusion_configs]
        )
    
    def forward(self, yolo_features, sam2_features):
        """
        yolo_features: YOLO backbone에서 추출한 각 scale의 feature map 리스트
        sam2_features: SAM2에서 추출한 각 scale의 feature map 리스트
        두 리스트의 순서가 동일한 scale 순서임을 가정합니다.
        """
        fused_features = []
        for fusion_block, yolo_feat, sam2_feat in zip(self.fusion_blocks, yolo_features, sam2_features):
            fused = fusion_block(yolo_feat, sam2_feat)
            fused_features.append(fused)
        return fused_features

# ----------------- 더미 YOLO 모델 정의 (예시) -----------------
# 실제로는 기존 YOLO 모델을 사용하지만, 여기서는 예시로 간단한 backbone과 detection head를 정의합니다.

class DummyYOLOBackbone(nn.Module):
    def __init__(self):
        super(DummyYOLOBackbone, self).__init__()
        # 예시로 3개의 convolution layer를 이용해 3개의 feature map 생성
        self.layer1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)   # 예상 output: [B, 32, 128, 128]
        self.layer2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 예상 output: [B, 64, 64, 64]
        self.layer3 = nn.Conv2d(64, 256, kernel_size=3, padding=1) # 예상 output: [B, 256, 32, 32]
    
    def forward(self, x):
        # layer1: feature map scale 0
        x0 = F.relu(self.layer1(x))
        # layer2: spatial downsampling을 위해 max pooling 후 feature map scale 1
        x1 = F.relu(self.layer2(F.max_pool2d(x0, 2)))
        # layer3: 다시 downsampling 후 feature map scale 2
        x2 = F.relu(self.layer3(F.max_pool2d(x1, 2)))
        # FPN과 유사하게 3개의 feature map을 반환
        return [x0, x1, x2]

class DummyYOLODetectionHead(nn.Module):
    def __init__(self):
        super(DummyYOLODetectionHead, self).__init__()
        # detection head 예시: fused feature map에서 예측을 위한 1x1 Conv
        self.conv = nn.Conv2d(256, 10, kernel_size=1)  # 예: 10개의 출력 채널
    
    def forward(self, fused_feature):
        return self.conv(fused_feature)

# ----------------- YOLO와 Fusion 모듈 통합 -----------------
class YOLOWithFusion(nn.Module):
    def __init__(self, fusion_module, yolo_backbone, detection_head):
        """
        fusion_module: YOLOSAM2Fusion 모듈 (여기서는 concat 방식)
        yolo_backbone: YOLO 모델의 backbone (FPN feature map 반환)
        detection_head: YOLO의 detection head (fused feature map을 입력받음)
        """
        super(YOLOWithFusion, self).__init__()
        self.yolo_backbone = yolo_backbone
        self.fusion_module = fusion_module
        self.detection_head = detection_head
    
    def forward(self, x, sam2_features):
        """
        x: 입력 이미지 tensor, shape: [B, 3, H, W]
        sam2_features: SAM2의 backbone_fpn feature map 리스트 (각 scale별로 3개)
        """
        # 1. YOLO backbone을 통해 FPN feature map 얻기
        yolo_features = self.yolo_backbone(x)
        
        # 2. YOLO feature와 SAM2 feature를 fusion 모듈로 융합 (여기서 concat 방식 사용)
        fused_features = self.fusion_module(yolo_features, sam2_features)
        
        # 3. 예시로, 가장 깊은 scale (채널 256인 feature map)을 detection head에 연결
        final_feature = fused_features[-1]
        predictions = self.detection_head(final_feature)
        return predictions

# ----------------- IPython Notebook에서 실행 예제 -----------------
if __name__ == "__main__":
    # 재현성을 위한 랜덤 시드 설정
    torch.manual_seed(42)
    
    # 1. Dummy YOLO backbone과 detection head 생성
    yolo_backbone = DummyYOLOBackbone()
    detection_head = DummyYOLODetectionHead()
    
    # 2. Fusion configuration: 각 scale별 (SAM2 채널, YOLO 채널)
    # 질문에서 주신 feature map 정보에 따라 설정
    fusion_configs = [(32, 32), (64, 64), (256, 256)]
    
    # 3. Fusion 모듈 초기화 (fusion_method는 'concat' 선택)
    fusion_module = YOLOSAM2Fusion(fusion_configs, fusion_method='concat')
    
    # 4. YOLO 모델과 Fusion 모듈을 통합한 모델 생성
    model = YOLOWithFusion(fusion_module, yolo_backbone, detection_head)
    
    # 5. Dummy 입력 이미지 생성 (예: 배치 크기 1, 3채널, 128x128 해상도)
    dummy_input = torch.randn(1, 3, 128, 128)
    
    # 6. SAM2에서 추출한 backbone_fpn feature map 생성 (각 scale별 dummy 데이터)
    sam2_feature0 = torch.randn(1, 32, 128, 128)   # Scale 0
    sam2_feature1 = torch.randn(1, 64, 64, 64)      # Scale 1
    sam2_feature2 = torch.randn(1, 256, 32, 32)     # Scale 2
    sam2_features = [sam2_feature0, sam2_feature1, sam2_feature2]
    
    # 7. 모델 forward pass 수행 (학습/추론 단계에서 사용)
    predictions = model(dummy_input, sam2_features)
    
    # 8. detection head의 예측 결과 출력
    print("Predictions shape:", predictions.shape)
    # 예시: [1, 10, 32, 32] (batch, channel, height, width)


Predictions shape: torch.Size([1, 10, 32, 32])
